# importing Reduerding Libreries

In [3]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import xgboost as xgb
import pickle
import time
import os
import joblib
import keyboard

# Load Model and LabelEncoder

In [4]:
import joblib

# Load XGBoost model
model = joblib.load('..//model//xgboost_model.pkl')

# Load LabelEncoder
label_encoder = joblib.load('..//model//label_encoder.pkl')

# Print classes
print("Model classes:", model.classes_ if hasattr(model, 'classes_') else "N/A")
print("LabelEncoder classes:", label_encoder.classes_)


Model classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
LabelEncoder classes: ['call' 'dislike' 'fist' 'four' 'like' 'mute' 'ok' 'one' 'palm' 'peace'
 'peace_inverted' 'rock' 'stop' 'stop_inverted' 'three' 'three2' 'two_up'
 'two_up_inverted']


# Initialize Webcam and Mediapipe

In [9]:
# Set up pyautogui
pyautogui.FAILSAFE = True  # Move mouse to top-left to stop
pyautogui.PAUSE = 0.01  # Small delay for smooth control

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Get screen size for cursor mapping
screen_width, screen_height = pyautogui.size()

# Define Function to Extract Hand Landmarks

In [10]:
def extract_hand_features(landmarks):
    
    # Get wrist and middle fingertip landmarks
    wrist = landmarks[0]
    middle_tip = landmarks[12]  # Landmark 12 is middle fingertip

    # Calculate scale (distance from wrist to middle fingertip)
    scale = np.sqrt(
        (middle_tip.x - wrist.x) ** 2 +
        (middle_tip.y - wrist.y) ** 2
    )
    
    # Prevent division by zero
    if scale == 0:
        scale = 1e-6

    features = []
    for lm in landmarks:
        x_norm = (lm.x - wrist.x) / scale
        y_norm = (lm.y - wrist.y) / scale
        z_norm = lm.z / scale  # Optional: normalize z only by scale
        features.extend([x_norm, y_norm, z_norm])
    
    return np.array(features)


def get_finger_status(hand_landmarks):
    """Returns binary status of each finger: [Thumb, Index, Middle, Ring, Pinky]"""
    tip_ids = [4, 8, 12, 16, 20]
    status = []

    # Thumb (horizontal check)
    if hand_landmarks[tip_ids[0]].x < hand_landmarks[tip_ids[0] - 1].x:
        status.append(1)
    else:
        status.append(0)

    # Other fingers (vertical check)
    for i in range(1, 5):
        if hand_landmarks[tip_ids[i]].y < hand_landmarks[tip_ids[i] - 2].y:
            status.append(1)
        else:
            status.append(0)

    return status


# Process Gestures and Map to Actions

In [11]:
# Confidence threshold
CONFIDENCE_THRESHOLD = 0.7

# Capture from camera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    h, w, _ = frame.shape

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            finger_status = get_finger_status(hand_landmarks.landmark)

            # ---------------------- Cursor Movement ----------------------
            if finger_status == [0, 1, 0, 0, 0]:  # Only index finger up
                index_tip = hand_landmarks.landmark[8]
                # Top of the file
                prev_cursor_x, prev_cursor_y = 0, 0

                # Inside loop
                smoothing_factor = 0.7  # closer to 1 = faster

                raw_x = int(np.clip(index_tip.x, 0, 1) * screen_width)
                raw_y = int(np.clip(index_tip.y, 0, 1) * screen_height)

                cursor_x = int(prev_cursor_x * (1 - smoothing_factor) + raw_x * smoothing_factor)
                cursor_y = int(prev_cursor_y * (1 - smoothing_factor) + raw_y * smoothing_factor)

                pyautogui.moveTo(cursor_x, cursor_y)
                prev_cursor_x, prev_cursor_y = cursor_x, cursor_y


                cv2.putText(frame, "Moving Cursor", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                continue

            

            # ---------------------- Model-Based Gesture ----------------------
            features = extract_hand_features(hand_landmarks.landmark).reshape(1, -1)
            gesture_probs = model.predict_proba(features)[0]
            gesture_idx = np.argmax(gesture_probs)
            confidence = gesture_probs[gesture_idx]
            gesture_label = label_encoder.inverse_transform([gesture_idx])[0]

            print(f"Gesture: {gesture_label}, Confidence: {confidence:.2f}")

            if confidence >= CONFIDENCE_THRESHOLD:
                if gesture_label == "stop":
                    pyautogui.click()
                    print("Action: Clicking")
                elif gesture_label == "peace":
                    pyautogui.hotkey('ctrl', '+')
                    print("Action: Zoom in")
                elif gesture_label == "peace_inverted":
                    pyautogui.hotkey('ctrl', '-')
                    print("Action: Zoom out")
                elif gesture_label == "dislike":
                    pyautogui.scroll(-100)
                    print("Action: Scroll down")
                elif gesture_label == "like":
                    pyautogui.scroll(100)
                    print("Action: Scroll up")
                elif gesture_label == "two_up":
                    pyautogui.mouseDown()
                    print("Action: Start drawing")
                elif gesture_label == "two_up_inverted":
                    pyautogui.mouseUp()
                    print("Action: Stop drawing")

    else:
        print("No hand detected")

    cv2.imshow("Hand Gesture Control", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Manual Quit")
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

No hand detected
No hand detected
No hand detected
No hand detected
Gesture: peace, Confidence: 0.44
Gesture: three2, Confidence: 0.67
Gesture: peace_inverted, Confidence: 0.91
Action: Zoom out
Gesture: three2, Confidence: 0.73
Gesture: peace_inverted, Confidence: 0.86
Action: Zoom out
Gesture: peace_inverted, Confidence: 0.72
Action: Zoom out
Gesture: peace_inverted, Confidence: 0.85
Action: Zoom out
Gesture: peace_inverted, Confidence: 0.49
Gesture: peace_inverted, Confidence: 0.76
Action: Zoom out
Gesture: three2, Confidence: 0.54
Gesture: peace_inverted, Confidence: 0.40
Gesture: peace_inverted, Confidence: 0.60
Gesture: three2, Confidence: 0.69
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No hand detected
No

KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Camera setup
cap = cv2.VideoCapture(0)
screen_width, screen_height = pyautogui.size()

# Gesture detection helpers
def get_finger_status(hand_landmarks):
    finger_up = []

    # Index finger
    if hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y:
        finger_up.append(1)
    else:
        finger_up.append(0)

    # Thumb
    if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
        finger_up.append(1)
    else:
        finger_up.append(0)

    return finger_up

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Mirror effect
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    h, w, _ = img.shape

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            finger_status = get_finger_status(hand_landmarks)

            index_finger_tip = hand_landmarks.landmark[8]
            x = int(index_finger_tip.x * w)
            y = int(index_finger_tip.y * h)

            screen_x = int(index_finger_tip.x * screen_width)
            screen_y = int(index_finger_tip.y * screen_height)

            if finger_status == [1, 0]:  # Only index finger up - Move cursor
                pyautogui.moveTo(screen_x, screen_y)
                cv2.circle(img, (x, y), 15, (255, 0, 0), cv2.FILLED)
                cv2.putText(img, "Moving Cursor", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            elif finger_status == [0, 1]:  # Only thumb up - Click
                pyautogui.click()
                cv2.putText(img, "Click!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                time.sleep(1)  # Prevent multiple clicks

    cv2.imshow("Smartboard Control", img)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 